In [1]:
import pandas as pd
import PyPDF2
from PIL import Image 
import pytesseract
from pdf2image import convert_from_path
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
import nltk

In [ ]:
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

In [ ]:
def textPytesseract(filename):
    
    #convert PDF to images
    # Store all the pages of the PDF in a variable 
    pages = convert_from_path(filename, 500) 
    
    image_counter = 1
    
    for page in pages: 
  
        # Declaring filename for each page of PDF as JPG 
        # For each page, filename will be: 
        # PDF page n -> page_n.jpg 
        filename = "page_"+str(image_counter)+".jpg"

        # Save the image of the page in system 
        page.save(filename, 'JPEG') 

        # Increment the counter to update filename 
        image_counter = image_counter + 1
    
    #Recognizing text from the images using OCR
    filelimit = image_counter-1
    text=""
    for i in range(1, filelimit + 1): 
    
        # Set filename to recognize text from 
        # Again, these files will be: 
        # page_n.jpg 
        filename = "page_"+str(i)+".jpg"

        # Recognize the text as string in image using pytesserct 
        text_ = str(((pytesseract.image_to_string(Image.open(filename))))) 

        # The recognized text is stored in variable text 
        # Any string processing may be applied on text 
        # Here, basic formatting has been done: 
        # In many PDFs, at line ending, if a word can't 
        # be written fully, a 'hyphen' is added. 
        # The rest of the word is written in the next line 
        # To remove this, we replace every '-\n' to ''. 
        text_ = text_.replace('-\n', '')
        text += text_
    
    return text.lower()

In [ ]:
def extract_phone_numbers(text):
    r = re.compile(r'(\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]??\d{4}|\d{3}[-\.\s]??\d{4})')
    phone_numbers = r.findall(text)
    return [re.sub(r'\D', '', number) for number in phone_numbers]

In [ ]:
def extract_email_addresses(text):
    r = re.compile(r'[\w\.-]+@[\w\.-]+')
    return r.findall(text)

In [ ]:
def ie_preprocess(text):
    stop_words = stopwords.words('english')
    document = ' '.join([i for i in text.split() if i not in stop_words])
    sentences = nltk.sent_tokenize(document)
    sentences = [nltk.word_tokenize(sent) for sent in sentences]
    sentences = [nltk.pos_tag(sent) for sent in sentences]
    return sentences

In [ ]:
def extract_names(text):
    names = []
    sentences = ie_preprocess(text)
    for tagged_sentence in sentences:
        for chunk in nltk.ne_chunk(tagged_sentence):
            if type(chunk) == nltk.tree.Tree:
                if chunk.label() == 'PERSON':
                    names.append(' '.join([c[0] for c in chunk]))
    return names

In [ ]:
text = textPytesseract('aditya bhartia (copy).pdf')
phone = extract_phone_numbers(text)
email = extract_email_addresses(text)
names = extract_names(text)

In [ ]:
print (phone)
print (email)
print (names)#need to train with Indian Names

In [ ]:
print (text)

In [ ]:
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()

In [ ]:
doc = nlp(text)

In [ ]:
print([(X.text, X.label_) for X in doc.ents])

In [ ]:
for X in doc.ents:
    if X.label_ == 'ORG':
        print (X.text)